Install required packages

In [ ]:
!pip install wandb

Import libraries

In [ ]:
import numpy as np
from keras.datasets import fashion_mnist
import wandb 
from sklearn.model_selection import train_test_split

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Loss functions, activation functions and their derivatives

In [ ]:
def _positive_sigmoid(x):
    return 1 / (1 + np.exp(-x))
def _negative_sigmoid(x):
    exp = np.exp(x)
    return exp / (exp + 1)

def sig(x):
    positive = x >= 0
    negative = ~positive
    result = np.empty_like(x)
    result[positive] = _positive_sigmoid(x[positive])
    result[negative] = _negative_sigmoid(x[negative])
    return result

def d_sig(x):
    s = sig(x)
    return (1 - s) * s

def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    return 1 - np.square(np.tanh(x))

def relu(z):
    return (z>0)*(z) + ((z<0)*(z)*0.01)

def der_relu(z):
    return (z>0)*np.ones(z.shape) + (z<0)*(0.01*np.ones(z.shape) )

def softmax(x):
    mx = np.max(x)
    z = x - mx + 500.0
    e_x = np.exp(z)
    return e_x / e_x.sum()

def soft_derv(y, x):     # y = target o/p (one-hot), x = preactivation value
    yh =  softmax(x)
    return (yh-y)
    
# Loss Functions 
def cross_enloss(y, yh):
    i = np.argmax(y)
    esp = 1e-300  
    loss = -np.log(yh[i]+esp)
    return loss[0]

def mse(y, yh):
    diff = np.subtract(y,yh)
    squared_diff = np.square(diff)
    return squared_diff.mean()

In [ ]:
#-- Helper Functions --------------------------
def oneHotEnc(Y, num_class):
    Yenc = np.zeros((num_class, int(Y.shape[0])))
    for i in range(Y.shape[0]):
        val = Y[i]
        Yenc[int(val)][i] = 1.0
    return Yenc

#### The layer class : initialize and maintain each layers' parameters

In [ ]:
# The layer class
class Layer:

    actFun = {
        'tanh': (tanh, d_tanh),
        'sigmoid': (sig, d_sig),
        'softmax': (softmax, soft_derv),
        'relu': (relu, der_relu)
        }
    

    def __init__(self, inputs, neurons, weight_initializer, activation, op=False):
        self.op = op
        self.W = np.random.randn(neurons, inputs)
        if weight_initializer == 'xavier':
            self.W = self.W/np.sqrt(inputs)
        self.b = np.random.randn(neurons,1)
        self.f, self.der = self.actFun.get(activation)
    
    def feed(self, inps):
        pA = np.matmul(self.W,inps) + self.b
        A = self.f(pA)
        return A
    
    def feedforward(self, inps):
        self.inps = inps
        self.preAct = np.matmul(self.W, self.inps) + self.b
        self.Act = self.f(self.preAct)
        return self.Act
    
    def calc_grad(self, delA, y = np.array([[]]) ):
        if self.op == True:
            delPA = self.der(y, self.preAct)
        else:
            delPA = np.multiply(self.der(self.preAct), delA)
        
        delPA = delPA.reshape(len(delPA), 1)
        dW = np.matmul(delPA, self.inps.T)
        db = delPA
        dA_prev = np.matmul(self.W.T, delPA) #derivative of error w.r.t to activations at previous layer
        
        return dW, db, dA_prev

####The network class : to initialize a neural network and use various optimizers

In [ ]:
class network:

    def __init__(self, n_layers, neurons, actfn, weight_initializer, X, Y, Xval, Yval):
        self.X = X
        self.Y = Y
        self.Xval = Xval
        self.Yval = Yval
        
        # initialize all the layers of the network
        self.layers = [ Layer( X.shape[0], neurons, weight_initializer, actfn) ]
        for i in range(1, n_layers):
            if i == (n_layers-1):
                self.layers.append( Layer(neurons, Y.shape[0], weight_initializer, 'softmax',True) )  #set layers of network
            else:
                self.layers.append( Layer(neurons, neurons, weight_initializer, actfn) )  #set layers of network

    # mini-batch gradient descent
    def batch(self, epochs=10, bat_size=16, loss_type = 'cross', l2 = 0, eta=0.01):
        losses = []
        for epoch in range(epochs):
            loss = 0.0
            DW = []     #List of W_grads of all layer
            DB = []     #List of b_grads of all layer
            for layer in self.layers:
                DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
                DB.append(np.zeros((layer.b.shape[0],1)) )  

            data_points = 0
            for x,y in zip(self.X.transpose(),self.Y.transpose()):
                #feedforward
                y = y.reshape(len(y),1)
                yh = x.reshape(len(x),1)
                
                for layer in self.layers:
                    yh = layer.feedforward(yh)
                
                delA = yh

                # gradient calculation
                for i in range(len(self.layers)-1, -1, -1):                 
                    if (self.layers[i].op==True):
                        dW,db,delA = self.layers[i].calc_grad(delA, y)
                    else:
                        dW,db,delA = self.layers[i].calc_grad(delA)

                    DW[i] = DW[i]+dW
                    DB[i] = DB[i]+db

                data_points += 1
                # mini-batch update ----------------------------------------
                if(data_points%bat_size==0):
                    for i in range(len(self.layers)-1, -1, -1):
                        nw = (eta*DW[i])/bat_size
                        nb = (eta*DB[i])/bat_size
                        self.layers[i].W =  (1-eta*l2)*self.layers[i].W - nw
                        self.layers[i].b =  self.layers[i].b - nb           
                    DW.clear()
                    DB.clear()
                    for layer in self.layers:
                        DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
                        DB.append(np.zeros((layer.b.shape[0],1)) ) 
            
            
            vacc, vloss = self.metric(self.Xval,self.Yval,loss_type)
            acc, loss = self.metric(self.X,self.Y,loss_type)
            losses.append(loss)
            wandb.log({"epoch": epoch+1, 
                       "loss": loss,
                       "accuracy": acc,
                       "val_error": vloss,
                       "val_accuracy": vacc})
            
            print("%d Complete. Loss = %f, acc = %f total loss=%f" %(epoch+1, loss, acc, vloss))

        print("Update complete. Total epochs = %d" %(epochs,))
        return losses
        
    # stochastic gradient descent    
    def sgd(self, epochs, loss_type = 'cross', l2 = 0, eta = 0.01):
        losses = []
        for epoch in range(epochs):
            loss = 0.0
            for x,y in zip(self.X.transpose(),self.Y.transpose()):
                #feedforward----------------------------------
                y = y.reshape(len(y),1)
                yh = x.reshape(len(x),1)
                for layer in self.layers:
                    yh = layer.feedforward(yh)
                
                delA = yh
                #Backpropagation---------------------------
                for layer in reversed(self.layers):
                    if(layer.op==True):
                        dW,db,delA = layer.calc_grad(delA, y)
                    else:
                        dW,db,delA = layer.calc_grad(delA)
                    # updates for each data point
                    layer.W = (1-eta*l2)*layer.W - (dW*eta)
                    layer.b = layer.b - (db*eta)

            vacc, vloss = self.metric(self.Xval,self.Yval,loss_type)
            acc, loss = self.metric(self.X,self.Y,loss_type)
            losses.append(loss)
            wandb.log({"epoch": epoch+1, 
                       "loss": loss,
                       "accuracy": acc,
                       "val_error": vloss,
                       "val_accuracy": vacc})
            
            print("%d Complete. Loss = %f, acc = %f total loss=%f" %(epoch+1, loss, acc, vloss))

        print("Update complete. Total epochs = %d" %(epochs,))
        return losses

    def momentum(self, epochs = 10, bat_size = 16, loss_type = 'cross', l2 = 0, eta = 0.01, gamma = 0.9):

        losses = []
        VW = []           #momentum parameter for W
        VB = []           #Momentum parameter for b
        data_points = 0
        DW = []     #List of W_grads of all layer
        DB = []     #List of b_grads of all layer
        for layer in self.layers:
            VW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
            VB.append(np.zeros((layer.b.shape[0],1)))
            DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
            DB.append(np.zeros((layer.b.shape[0],1)) ) 

        for epoch in range(epochs):
            loss = 0.0
            for x,y in zip(self.X.transpose(),self.Y.transpose()):
                #feedforward ----------------------------------
                data_points += 1
                y = y.reshape(len(y),1)
                yh = x.reshape(len(x),1)
                for layer in self.layers:
                    yh = layer.feedforward(yh)

                delA = yh

                # gradient calculation ----------------------------
                for i in range(len(self.layers)-1, -1, -1):                   
                    if(self.layers[i].op==True):
                        dW,db,delA = self.layers[i].calc_grad(delA, y)
                    else:
                        dW,db,delA = self.layers[i].calc_grad(delA)

                    DW[i] = DW[i]+dW
                    DB[i] = DB[i]+db
                       
                # MomentumGD update ----------------------------------------
                if(data_points==bat_size):
                    data_points=0
                    for i in range(len(self.layers)-1, -1, -1):
                        VW[i] = (gamma*VW[i])+ ((eta*DW[i])/bat_size)
                        VB[i] = (gamma*VB[i])+ ((eta*DB[i])/bat_size)
                        self.layers[i].W =  (1-eta*l2)*self.layers[i].W - VW[i]
                        self.layers[i].b =  self.layers[i].b - VB[i]
                    DW.clear()
                    DB.clear()
                    for layer in self.layers:
                        DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
                        DB.append(np.zeros((layer.b.shape[0],1)) ) 


            vacc,vloss = self.metric(self.Xval,self.Yval,loss_type)
            acc,loss = self.metric(self.X,self.Y,loss_type)
            losses.append(loss)
            wandb.log({"epoch": epoch+1, 
                       "loss": loss,
                       "accuracy": acc,
                       "val_error": vloss,
                       "val_accuracy": vacc})
            
            print("%d Complete. Loss = %f, acc = %f total loss=%f" %(epoch+1, loss, acc, vloss))

        print("Update complete. Total epochs = %d" %(epochs,))
        return losses
    
    # nasterov accelerated gradient descent
    def nag(self, epochs = 10, bat_size = 16, loss_type = 'cross', l2 = 0, eta = 0.01, gamma = 0.9):
        losses = []
        pVW = []          # prev momentum parameter for W
        pVB = []          # prev Momentum parameter for b
        VW = []           # momentum parameter for W
        VB = []           # Momentum parameter for b
        W_old =[]
        b_old = []
        for layer in self.layers:
            VW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
            VB.append(np.zeros((layer.b.shape[0],1)))
            pVW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
            pVB.append(np.zeros((layer.b.shape[0],1)))
            W_old.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
            b_old.append(np.zeros((layer.b.shape[0],1)))

        data_points = 0
        DW = []     #List of W_grads of all layer
        DB = []     #List of b_grads of all layer
        for i in range(len(self.layers)):
            DW.append(np.zeros((self.layers[i].W.shape[0], self.layers[i].W.shape[1] )) )
            DB.append(np.zeros((self.layers[i].b.shape[0],1)) )
            W_old[i] = self.layers[i].W
            b_old[i] = self.layers[i].b

        for epoch in range(epochs):
            loss = 0.0
            for x,y in zip(self.X.transpose(),self.Y.transpose()):
                #feedforward ----------------------------------
                data_points += 1
                y = y.reshape(len(y),1)
                yh = x.reshape(len(x),1)
                
                for layer in self.layers:
                    yh = layer.feedforward(yh)

                delA = yh

                # gradient calculation ----------------------------
                for i in range(len(self.layers)-1, -1, -1):                   
                    if(self.layers[i].op==True):
                        dW,db,delA = self.layers[i].calc_grad(delA, y)
                    else:
                        dW,db,delA = self.layers[i].calc_grad(delA)

                    DW[i] = DW[i]+dW
                    DB[i] = DB[i]+db
                       
                # NAG updates ----------------------------------------
                if(data_points==bat_size):
                    data_points=0
                    for i in range(len(self.layers)-1, -1, -1):
                        VW[i] = (gamma*pVW[i])+ ((eta*DW[i])/bat_size)
                        VB[i] = (gamma*pVB[i])+ ((eta*DB[i])/bat_size)
                        
                        self.layers[i].W =  (1-eta*l2)*W_old[i] - gamma*pVW[i]
                        self.layers[i].b =  b_old[i] - gamma*pVB[i]
                        
                        W_old[i] =  W_old[i] - VW[i]
                        b_old[i] =  b_old[i] - VB[i]
                    
                    pVW = VW
                    pVB = VB
                        
                        
                    DW.clear()
                    DB.clear()
                    for layer in self.layers:
                        DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )) )
                        DB.append(np.zeros((layer.b.shape[0],1)) )

            vacc,vloss = self.metric(self.Xval,self.Yval,loss_type)
            acc,loss = self.metric(self.X,self.Y,loss_type)
            losses.append(loss)
            wandb.log({"epoch": epoch+1, 
                       "loss": loss,
                       "accuracy": acc,
                       "val_error": vloss,
                       "val_accuracy": vacc})
            
            print("%d Complete. Loss = %f, acc = %f total loss=%f" %(epoch+1, loss, acc, vloss))
            
        for i in range(len(self.layers)):
            self.layers[i].W =  W_old[i]
            self.layers[i].b =  b_old[i]
            
        print("Update complete. Total epochs = %d" %(epochs,))
        return losses

    # RMSProp
    def rmsprop(self, epochs = 10, batch_size = 16, loss_type = 'cross', l2 = 0, eta = 0.01, beta = 0.9, epsilon = 1e-8):
            
        epsW = []
        epsB = []
        nW = []
        nB = []
        losses = []
        DW = []     #List of W_grads of all layer
        DB = []     #List of b_grads of all layer
        seen = 0

        for layer in self.layers:
            epsW.append(np.full((layer.W.shape[0], layer.W.shape[1]),epsilon))
            epsB.append(np.full((layer.b.shape[0],1),epsilon))
            nW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            nB.append(np.zeros((layer.b.shape[0],1)))
            DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            DB.append(np.zeros((layer.b.shape[0],1)))

        for epoch in range(epochs):
            loss = 0.0
            for x,y in zip(self.X.transpose(),self.Y.transpose()):
                seen += 1
                #feedforward
                y = y.reshape(len(y),1)
                yh = x.reshape(len(x),1)
                
                for layer in self.layers:
                    yh = layer.feedforward(yh)
               
                delA = yh

                #gradient calculation
                for i in range(len(self.layers)-1, -1, -1):                 
                    if (self.layers[i].op==True):
                        dW,db,delA = self.layers[i].calc_grad(delA, y)
                    else:
                        dW,db,delA = self.layers[i].calc_grad(delA)

                    DW[i] = DW[i]+dW
                    DB[i] = DB[i]+db

                if (seen == batch_size):
                    DW[i] = DW[i]/batch_size
                    DB[i] = DB[i]/batch_size
                    seen = 0
                    #rmsprop update
                    for i in range(len(self.layers)-1, -1, -1):
                        nW[i] = (beta*nW[i]) + (1 - beta)*(np.square(DW[i]))
                        nB[i] = (beta*nB[i]) + (1 - beta)*(np.square(DB[i]))

                        self.layers[i].W =  (1-eta*l2)*self.layers[i].W - np.multiply((eta/np.sqrt(nW[i] + epsW[i])),DW[i])
                        self.layers[i].b =  self.layers[i].b - np.multiply((eta/np.sqrt(nB[i] + epsB[i])),DB[i]) 
                    DW.clear()
                    DB.clear()
                    for layer in self.layers:
                        DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )))
                        DB.append(np.zeros((layer.b.shape[0],1))) 

            vacc,vloss = self.metric(self.Xval,self.Yval,loss_type)
            acc, loss = self.metric(self.X,self.Y,loss_type)
            losses.append(loss)
            '''wandb.log({"epoch": epoch+1, 
                       "loss": loss,
                       "accuracy": acc,
                       "val_error": vloss,
                       "val_accuracy": vacc})'''
            
            print("%d Complete. Loss = %f, acc = %f total loss=%f" %(epoch+1, loss, acc, vloss))

        print("Update complete. Total epochs = %d" %(epochs,))
        return losses

    #Adam 
    def adam(self, epochs = 10, batch_size = 16, loss_type = 'cross', l2 = 0, eta = 0.01, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8):
        epsW = []
        epsB = []
        vW = []
        vB = []
        mW = []
        mB = []
        losses = []
        DW = []     #List of W_grads of all layer
        DB = []     #List of b_grads of all layer
        seen = 0

        for layer in self.layers:
            epsW.append(np.full((layer.W.shape[0], layer.W.shape[1]),epsilon))
            epsB.append(np.full((layer.b.shape[0],1),epsilon))
            mW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            mB.append(np.zeros((layer.b.shape[0],1)))
            vW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            vB.append(np.zeros((layer.b.shape[0],1)))
            DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            DB.append(np.zeros((layer.b.shape[0],1)))

        for epoch in range(epochs):
            loss = 0.0
            for x,y in zip(self.X.transpose(),self.Y.transpose()):
                seen += 1
                #feedforward
                y = y.reshape(len(y),1)
                yh = x.reshape(len(x),1)
                
                for layer in self.layers:
                    yh = layer.feedforward(yh)
                
                delA = yh

                #gradient calculation
                for i in range(len(self.layers)-1, -1, -1):                 
                    if (self.layers[i].op==True):
                        dW,db,delA = self.layers[i].calc_grad(delA, y)
                    else:
                        dW,db,delA = self.layers[i].calc_grad(delA)

                    DW[i] = DW[i]+dW
                    DB[i] = DB[i]+db

                if (seen == batch_size):
                    seen = 0
                    DW[i] = DW[i]/batch_size
                    DB[i] = DB[i]/batch_size
                    #adam update
                    f1 = 1 - (beta1**(epoch+1))
                    f2 = 1 - (beta2**(epoch+1))
                    for i in range(len(self.layers)-1, -1, -1):
                        mW[i] = (beta1 * mW[i]) + (1 - beta1)*DW[i]
                        mB[i] = (beta1 * mB[i]) + (1 - beta1)*DB[i]
                        vW[i] = (beta2 * vW[i]) + (1 - beta2)*(np.square(DW[i]))
                        vB[i] = (beta2 * vB[i]) + (1 - beta2)*(np.square(DB[i]))
                        
                        self.layers[i].W =  (1-eta*l2)*self.layers[i].W - np.multiply((eta/np.sqrt((vW[i] / f2) + epsW[i])),mW[i]/ f1)
                        self.layers[i].b =  self.layers[i].b - np.multiply((eta/np.sqrt((vB[i]/ f2) + epsB[i])),mB[i]/ f1) 
                    DW.clear()
                    DB.clear()
                    for layer in self.layers:
                        DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )))
                        DB.append(np.zeros((layer.b.shape[0],1))) 


            vacc, vloss = self.metric(self.Xval,self.Yval,loss_type)
            acc, loss = self.metric(self.X,self.Y,loss_type)
            losses.append(loss)
            wandb.log({"epoch": epoch+1, 
                       "loss": loss,
                       "accuracy": acc,
                       "val_error": vloss,
                       "val_accuracy": vacc})
            
            print("%d Complete. Loss = %f, acc = %f total loss=%f" %(epoch+1, loss, acc, vloss))

        print("Update complete. Total epochs = %d" %(epochs,))
        return losses

    #Nadam
    def nadam(self, epochs = 10, batch_size = 16, loss_type = 'cross', l2 = 0, eta = 0.01, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8):    
        epsW = []
        epsB = []
        vW = []
        vB = []
        mW = []
        mB = []
        losses = []
        DW = []     #List of W_grads of all layer
        DB = []     #List of b_grads of all layer
        seen = 0

        for layer in self.layers:
            epsW.append(np.full((layer.W.shape[0], layer.W.shape[1]),epsilon))
            epsB.append(np.full((layer.b.shape[0],1),epsilon))
            mW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            mB.append(np.zeros((layer.b.shape[0],1)))
            vW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            vB.append(np.zeros((layer.b.shape[0],1)))
            DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1])))
            DB.append(np.zeros((layer.b.shape[0],1)))

        for epoch in range(epochs):
            loss2 = 0.0
            for x,y in zip(self.X.transpose(),self.Y.transpose()):
                seen += 1
                #feedforward
                y = y.reshape(len(y),1)
                yh = x.reshape(len(x),1)
                
                for layer in self.layers:
                    yh = layer.feedforward(yh)

                delA = yh

                #gradient calculation
                for i in range(len(self.layers)-1, -1, -1):                 
                    if (self.layers[i].op==True):
                        dW,db,delA = self.layers[i].calc_grad(delA, y)
                    else:
                        dW,db,delA = self.layers[i].calc_grad(delA)

                    DW[i] = DW[i]+dW
                    DB[i] = DB[i]+db

                if (seen == batch_size):
                    seen = 0
                    DW[i] = DW[i]/batch_size
                    DB[i] = DB[i]/batch_size
                    #nadam update
                    f1 = 1 - (beta1**(epoch+1))
                    f2 = 1 - (beta2**(epoch+1))
                    for i in range(len(self.layers)-1, -1, -1):
                        mW[i] = (beta1 * mW[i]) + (1 - beta1)*DW[i]
                        mB[i] = (beta1 * mB[i]) + (1 - beta1)*DB[i]
                        vW[i] = (beta2 * vW[i]) + (1 - beta2)*(np.square(DW[i]))
                        vB[i] = (beta2 * vB[i]) + (1 - beta2)*(np.square(DB[i]))
                        
                        self.layers[i].W =  (1-eta*l2)*self.layers[i].W - np.multiply((eta/np.sqrt((vW[i] * beta2 / f2) + epsW[i])),((mW[i] * beta1) + (1-beta1)*DW[i])/f1)
                        self.layers[i].b =  self.layers[i].b - np.multiply((eta/np.sqrt((vB[i] * beta2 / f2) + epsB[i])), ((mB[i] * beta1) + (1-beta1)*DB[i])/f1)
                    DW.clear()
                    DB.clear()
                    for layer in self.layers:
                        DW.append(np.zeros((layer.W.shape[0], layer.W.shape[1] )))
                        DB.append(np.zeros((layer.b.shape[0],1))) 

            vacc,vloss = self.metric(self.Xval,self.Yval,loss_type)
            acc, loss = self.metric(self.X,self.Y,loss_type)
            losses.append(loss)
            wandb.log({"epoch": epoch+1, 
                       "loss": loss,
                       "accuracy": acc,
                       "val_error": vloss,
                       "val_accuracy": vacc})
            
            print("%d Complete. Loss = %f, acc = %f total loss=%f" %(epoch+1, loss, acc, vloss))

        print("Update complete. Total epochs = %d" %(epochs,))
        return losses

    # to calculate the predicted y value
    def classify(self, X):
        pred = []
        for x in X.transpose():
            yh = x.reshape(len(x),1)
            
            #feeding forward without saving
            for layer in self.layers:
                yh = layer.feed(yh)
                
            pred.append(yh.T)
        
        return pred

    # to calculate accuracy and loss
    def metric(self,x,y,loss_type):
        pred = self.classify(x)
        count = 0
        for i in range(y.shape[1]):
            if(np.argmax(pred[i]) == np.argmax(y.T[i])):
                count += 1
        acc = (count*100)/y.shape[1]
        loss = 0
        if loss_type == 'mse':
            for i in range(0, y.shape[1]):
                loss = loss + mse(y.T[i],pred[i].T)
        else:
            for i in range(0, y.shape[1]):
                index = np.argmax(y.T[i])
                loss = loss + (-np.log((1e-300)+float(pred[i][0][index])))
        loss = loss/y.shape[1]
        return acc, loss

#### Data generation and pre-processing

In [ ]:
def data_generation():
    # load dataset
    (trainX, trainy), (testX, testy) = fashion_mnist.load_data()
    trainX = trainX.reshape(trainX.shape[0], 784)
    testX = testX.reshape(testX.shape[0], 784)

    trainX = trainX/255.0
    testX = testX/255.0

    X_train, X_val, y_train, y_val = train_test_split(trainX, trainy, test_size=0.1, random_state=45)
    X_train = X_train.transpose()
    X_val = X_val.transpose()
    ytrain_oneHot = oneHotEnc(y_train,10)
    yval_oneHot = oneHotEnc(y_val,10)
    return X_train, ytrain_oneHot, X_val, yval_oneHot

#### Setup sweep configurations for wandb 

In [ ]:
sweep_config = {
    'name' : 'WandB sweep',
    "method": 'random', #grid
    'metric': { 
        'name':'val_accuracy',
        'goal': 'maximize',
        },
    'parameters':{
        'epochs' : {'values' : [5, 10]},
        'no_hidden_layers' : {'values' : [3,4,5]},
        'weight_decay' : {'values' : [0, 0.0005, 0.5]},
        'neurons' : {'values' : [32,64,128]},
        'learning_rate': {'values' : [.001,0.0001]},
        'optimizer':{'values' : ['sgd','momentumgd','NAG','rmsprop','adam','nadam']},
        'batch_size' : {'values': [16,32,64]},
        'loss_func':{'values' : [ 'cross_entropy']},
        'weight_initializer':{'values' : ['random','xavier']},
        'activation' : {'values' : ['tanh','sigmoid','relu']},
        }
}

#### Train the model using the sweep config

In [ ]:
def train ():
    config_defaults = {
        'epochs': 10,
        'no_hidden_layers' : 4,
        'weight_decay' : 0.0005,
        'neurons' : 16,
        'learning_rate': 0.001,
        'optimizer':'nadam',
        'batch_size' : 16,
        'loss_func': 'cross_entropy',
        'weight_initializer': 'random',
        'activation' : 'sigmoid',
        }
    
    wandb.init(config=config_defaults)
    config = wandb.config

    wandb.run.name = "ep_{}_nhl_{}_hls_{}_eta_{}_opt_{}_bs_{}_loss_{}_winit_{}_act_{}_wdecay_{}".format(config.epochs,\
                                                                      config.no_hidden_layers,\
                                                                      config.neurons,\
                                                                      config.learning_rate,\
                                                                      config.optimizer,\
                                                                      config.batch_size,\
                                                                      config.loss_func,\
                                                                      config.weight_initializer,\
                                                                      config.activation,\
                                                                      config.weight_decay)
    
    X_train, ytrain_oneHot, X_val, y_val = data_generation()
    
    fist = network(config.no_hidden_layers+1, config.neurons, config.activation, config.weight_initializer ,X_train, ytrain_oneHot, X_val, y_val,)
    if config.optimizer == 'adam':
        losses = fist.adam(config.epochs, config.batch_size, config.loss_func, config.weight_decay, config.learning_rate)
    if config.optimizer == 'nadam':
        losses = fist.nadam(config.epochs, config.batch_size, config.loss_func, config.weight_decay, config.learning_rate)
    if config.optimizer == 'rmsprop':
        losses = fist.rmsprop(config.epochs, config.batch_size, config.loss_func, config.weight_decay, config.learning_rate)
    if config.optimizer == 'NAG':
        losses = fist.nag(config.epochs, config.batch_size, config.loss_func, config.weight_decay, config.learning_rate)
    if config.optimizer == 'momentumgd':
        losses = fist.momentum(config.epochs, config.batch_size, config.loss_func, config.weight_decay, config.learning_rate)
    if config.optimizer == 'sgd':
        losses = fist.sgd(config.epochs, config.loss_func, config.weight_decay, config.learning_rate)

    return losses


#### Get the sweep id and call the train function via wandb

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Ass1_sweep")

Create sweep with ID: 4w1yusga
Sweep URL: https://wandb.ai/pandp/Ass1_sweep/sweeps/4w1yusga


In [ ]:
wandb.agent(sweep_id, train, count = 40)

wandb: Agent Starting Run: 1en8u6zq with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	loss_func: cross_entropy
wandb: 	neurons: 64
wandb: 	no_hidden_layers: 3
wandb: 	optimizer: momentumgd
wandb: 	weight_decay: 0
wandb: 	weight_initializer: xavier


1 Complete. Loss = 0.580728, acc = 79.155556 total loss=0.576575
2 Complete. Loss = 0.494764, acc = 82.174074 total loss=0.495142
3 Complete. Loss = 0.455553, acc = 83.590741 total loss=0.459375
4 Complete. Loss = 0.432554, acc = 84.142593 total loss=0.441739
5 Complete. Loss = 0.409650, acc = 85.137037 total loss=0.422368
Update complete. Total epochs = 5


accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▄▃▂▁
val_accuracy,▁▅▆▇█
val_error,█▄▃▂▁
accuracy,85.13704
epoch,5
loss,0.40965
val_accuracy,84.91667
val_error,0.42237


wandb: Agent Starting Run: j3rrulhz with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: 	loss_func: cross_entropy
wandb: 	neurons: 128
wandb: 	no_hidden_layers: 4
wandb: 	optimizer: NAG
wandb: 	weight_decay: 0
wandb: 	weight_initializer: random


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in matmul


...

accuracy,▁▁▁▁▁
epoch,▁▃▅▆█
loss,█▆▄▃▁
val_accuracy,▁▁▁▁▁
val_error,█▆▄▃▁
accuracy,10.02222
epoch,5
loss,2.30317
val_accuracy,9.8
val_error,2.30394


wandb: Agent Starting Run: b5htpv1b with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0001
wandb: 	loss_func: cross_entropy
wandb: 	neurons: 128
wandb: 	no_hidden_layers: 5
wandb: 	optimizer: adam
wandb: 	weight_decay: 0
wandb: 	weight_initializer: random


1 Complete. Loss = 350.740889, acc = 49.079630 total loss=351.708039
2 Complete. Loss = 311.310760, acc = 54.790741 total loss=315.988158
3 Complete. Loss = 290.246988, acc = 57.820370 total loss=294.113487
4 Complete. Loss = 275.638990, acc = 59.946296 total loss=277.496523
5 Complete. Loss = 264.448655, acc = 61.505556 total loss=264.508209
Update complete. Total epochs = 5


accuracy,▁▄▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁▄▆▇█
val_error,█▅▃▂▁
accuracy,61.50556
epoch,5
loss,264.44865
val_accuracy,61.51667
val_error,264.50821


wandb: Agent Starting Run: 5ptk0y83 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	loss_func: cross_entropy
wandb: 	neurons: 32
wandb: 	no_hidden_layers: 4
wandb: 	optimizer: momentumgd
wandb: 	weight_decay: 0.5
wandb: 	weight_initializer: xavier


#### Plotting confusion matrix using wandb

In [29]:
# Reference: https://github.com/zalandoresearch/fashion-mnist/blob/master/README.md#Labels
labels = ['Top','Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag',
			   'Ankle boot']

#Confusion Matrix
def cm_show():
    wandb.init()
    X_train, ytrain_oneHot, X_val, y_val = data_generation()
    fist = network(6, 64, 'tanh', 'xavier' ,X_train, ytrain_oneHot, X_val, y_val)
    losses = fist.rmsprop(5, 64, 'cross', 0.5, 0.0001)
    yh = fist.classify(X_val)
    y_pred = []
    y_true = []
    for i in range(len(yh)):
        y_pred.append(np.argmax(yh[i]))
        y_true.append(np.argmax(y_val.T[i]))
    
    wandb.log({"ep=5_nhl=5_hls=64_eta=.0001_opt='rmsprop'_bs=64_loss='cross'_winit='xavier'_act='tanh'_wdecay=0.5" : wandb.sklearn.plot_confusion_matrix(y_true, y_pred, labels)})
    

In [30]:
cm_show()

1 Complete. Loss = 0.669864, acc = 77.898148 total loss=0.669412
2 Complete. Loss = 0.501574, acc = 82.559259 total loss=0.499425
3 Complete. Loss = 0.450500, acc = 84.166667 total loss=0.451900
4 Complete. Loss = 0.430050, acc = 84.707407 total loss=0.435804
5 Complete. Loss = 0.406261, acc = 85.650000 total loss=0.413926
Update complete. Total epochs = 5
